In [1]:
%matplotlib inline

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os
from tqdm.auto import tqdm
import shutil
import random
import xml.etree.ElementTree as ET
from xml.dom import minidom
from pathlib import Path
import glob


import torch
import torchvision

from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets, utils
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import json

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import sklearn.metrics as met
import scipy.io

random.seed(42)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Importing Dataset

In [3]:
!cp /content/gdrive/Shareddrives/BootCamp/Proyecto/dataset_YOLOV5.tar.gz .
!tar -xf /content/dataset_YOLOV5.tar.gz

In [4]:
!mv /content/content/dataset .
!rm -r /content/content

## Cloning YOLOV5

In [5]:
# Clone the model 
!git clone https://github.com/ultralytics/yolov5  

Cloning into 'yolov5'...
remote: Enumerating objects: 12496, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 12496 (delta 24), reused 26 (delta 9), pack-reused 12448
Receiving objects: 100% (12496/12496), 12.24 MiB | 24.43 MiB/s, done.
Resolving deltas: 100% (8616/8616), done.


In [7]:
# Installing requirements
!pip install -U -r yolov5/requirements.txt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-4.21.2-cp37-abi3-manylinux2014_x86_64.whl (407 kB)


## Checking YOLOV5 architecture

In [8]:
# Access the yolov5 folder and show the model's architecture
%cd /content/yolov5
%cat /content/yolov5/models/yolov5s.yaml

/content/yolov5
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4],

In [9]:
# Code to write template
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [10]:
%%writetemplate /content/yolov5/data_cells.yaml

train: ../dataset/train/images
val: ../dataset/valid/images
test: ../dataset/test/images

# Number of Classes
nc: 5

# Class Names
names: ['Superficial-Intermediate', 'Parabasal', 'Metaplastic', 'Koilocytotic', 'Dyskeratotic']

In [11]:
# Reading created template
%cat /content/yolov5/data_cells.yaml


train: ../dataset/train/images
val: ../dataset/valid/images
test: ../dataset/test/images

# Number of Classes
nc: 5

# Class Names
names: ['Superficial-Intermediate', 'Parabasal', 'Metaplastic', 'Koilocytotic', 'Dyskeratotic']

In [12]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: 5  # number of classes 
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [13]:
# Reading created template
%cat /content/yolov5/models/custom_yolov5s.yaml


# parameters
nc: 5  # number of classes 
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[

## Training the model

In [14]:
# check if we're using torch with GPU
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.12.0+cu113 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [15]:
#!rm -r /content/yolov5/runs/train/yolov5s_results

In [16]:
%cd /content/yolov5/
!python train.py --img 1000 --batch 32 --epochs 500 --data './data_cells.yaml' --cfg './models/custom_yolov5s.yaml' --weights '' --name yolov5s_results --cache

/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=./data_cells.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=32, imgsz=1000, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-294-g574ceed Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

## Saving Results

In [20]:
%cd /content/
!tar -czvf yoloV5.tar.gz /content/yolov5

/content
tar: Removing leading `/' from member names
/content/yolov5/
/content/yolov5/.pre-commit-config.yaml
/content/yolov5/tutorial.ipynb
/content/yolov5/export.py
/content/yolov5/CONTRIBUTING.md
/content/yolov5/__pycache__/
/content/yolov5/__pycache__/val.cpython-37.pyc
/content/yolov5/data_cells.yaml
/content/yolov5/hubconf.py
/content/yolov5/val.py
/content/yolov5/LICENSE
/content/yolov5/data/
/content/yolov5/data/SKU-110K.yaml
/content/yolov5/data/images/
/content/yolov5/data/images/zidane.jpg
/content/yolov5/data/images/bus.jpg
/content/yolov5/data/coco.yaml
/content/yolov5/data/scripts/
/content/yolov5/data/scripts/get_coco128.sh
/content/yolov5/data/scripts/get_coco.sh
/content/yolov5/data/scripts/download_weights.sh
/content/yolov5/data/hyps/
/content/yolov5/data/hyps/hyp.scratch-low.yaml
/content/yolov5/data/hyps/hyp.scratch-med.yaml
/content/yolov5/data/hyps/hyp.VOC.yaml
/content/yolov5/data/hyps/hyp.Objects365.yaml
/content/yolov5/data/hyps/hyp.scratch-high.yaml
/content/

In [21]:
!cp /content/yoloV5.tar.gz /content/gdrive/Shareddrives/BootCamp/Proyecto